# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **FPL player score** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, combine predictions with static data
3. Store predictions in a monitoring feature group

## <span style='color:#ff5f27'> 📝 Imports

In [419]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from functions import util
import os
from importlib import reload
import requests

In [420]:
reload(util)

print(util.get_gameweek_from_date("2025-1-4"))

20


## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [421]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
fs = project.get_feature_store()

2025-01-07 13:08:00,823 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 13:08:00,889 INFO: Initializing external client
2025-01-07 13:08:00,889 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 13:08:02,315 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159321


## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [422]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="player_score_xgboost_model",
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2025-01-07 13:08:03,874 WARNING: VersionWarning: No version provided for getting model `player_score_xgboost_model`, defaulting to `1`.



In [423]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='1.5788268E0', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'floa...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [424]:
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
general_info = requests.get(bootstrap_url).json()

In [425]:
try:
    monitor_fg = fs.get_feature_group(
            name='fpl_predictions',
        )
    prev_gameweek_nr = int(monitor_fg.select(["gameweek"]).read().max())
except:
    print("Feature group not found, use player features instead")
    player_fg = fs.get_feature_group("player_features")
    prev_gameweek_nr = int(player_fg.select(["gameweek"]).read().max()) - 1

2025-01-07 13:08:06,481 WARNING: VersionWarning: No version provided for getting feature group `fpl_predictions`, defaulting to `1`.

Feature group not found, use player features instead
2025-01-07 13:08:06,699 WARNING: VersionWarning: No version provided for getting feature group `player_features`, defaulting to `1`.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.89s) 
2025-01-07 13:08:09,219 WARNING: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



In [426]:
prev_gameweek_nr

17

In [427]:
prev_gameweek = next(
    (gw for gw in general_info['events'] if gw['id'] == prev_gameweek_nr), None
)

prev_finished = prev_gameweek['finished']
prev_finished

True

## <span style="color:#ff5f27;">✨ Get Player Score Features From Previous Gameweek </span>



In [428]:
if prev_finished:
    players_fg = fs.get_feature_group(
        name='player_features',
        version=1,
    )
    players = players_fg.filter(players_fg.gameweek == prev_gameweek_nr + 1).read()
    print("get stats for finished gameweek")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.86s) 
get stats for finished gameweek


In [429]:
players

id  points  gameweek  prev_minutes  prev_goals_scored  prev_assists  \
0    434       3        18          90.0                0.0           0.0   
1     79       0        18           0.0                0.0           0.0   
2    224       2        18          73.0                0.0           0.0   
3    319       1        18           3.0                0.0           0.0   
4    525       3        18          90.0                1.0           0.0   
..   ...     ...       ...           ...                ...           ...   
479  142       0        18           0.0                0.0           0.0   
480  486       1        18          90.0                0.0           0.0   
481  117       1        18           6.0                0.0           0.0   
482  184       0        18           0.0                0.0           0.0   
483  472       0        18           0.0                0.0           0.0   

     prev_clean_sheets  prev_goals_conceded  prev_own_goals  \
0                  1.0                  0.0             0.0   
1                  0.0                  0.0             0.0   
2                  1.0                  0.0             0.0   
3                  0.0                  0.0             0.0   
4                  0.0                  1.0             0.0   
..                 ...                  ...             ...   
479                0.0                  0.0             0.0   
480                0.0                  6.0             0.0   
481                0.0                  0.0             0.0   
482                0.0                  0.0             0.0   
483                0.0                  0.0             0.0   

     prev_penalties_saved  ...  prev_expected_goals  prev_expected_assists  \
0                     0.0  ...                 0.11                   0.09   
1                     0.0  ...                 0.00                   0.00   
2                     0.0  ...                 0.56                   0.00   
3                     0.0  ...                 0.00                   0.00   
4                     0.0  ...                 0.23                   0.01   
..                    ...  ...                  ...                    ...   
479                   0.0  ...                 0.00                   0.00   
480                   0.0  ...                 0.00                   0.01   
481                   0.0  ...                 0.00                   0.00   
482                   0.0  ...                 0.00                   0.00   
483                   0.0  ...                 0.00                   0.00   

     prev_expected_goal_involvements  prev_expected_goals_conceded  \
0                               0.20                          0.72   
1                               0.00                          0.00   
2                               0.56                          0.80   
3                               0.00                          0.13   
4                               0.24                          0.94   
..                               ...                           ...   
479                             0.00                          0.00   
480                             0.01                          4.60   
481                             0.00                          0.06   
482                             0.00                          0.00   
483                             0.00                          0.00   

     prev_in_dreamteam  prev_total_points  prev_selected  \
0                  0.0               53.0       189557.0   
1                  0.0               21.0        44981.0   
2                  0.0               30.0        11893.0   
3                  0.0                4.0        24273.0   
4                  0.0               44.0       277103.0   
..                 ...                ...            ...   
479                0.0                9.0       118315.0   
480                0.0               20.0        43379.0   
481   

### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [430]:
inputs = ['prev_minutes', 'prev_goals_scored', 'prev_assists',
         'prev_clean_sheets', 'prev_goals_conceded', 'prev_own_goals',
         'prev_penalties_saved', 'prev_penalties_missed', 'prev_yellow_cards',
         'prev_red_cards', 'prev_saves', 'prev_bonus',
         'prev_bps', 'prev_influence', 'prev_creativity',
         'prev_threat', 'prev_ict_index', 'prev_starts',
         'prev_expected_goals', 'prev_expected_assists', 'prev_expected_goal_involvements',
         'prev_expected_goals_conceded', 'prev_in_dreamteam', 'prev_total_points',
         'prev_selected', 'prev_transfers_balance', 'prev_value', 'prev_was_home', 'prev_opponent_team',]

#for i in range(0, len(input_data)):
#    output_data.at[i, 'predicted_score'] = retrieved_xgboost_model.predict(input_data.loc[i, inputs].values.reshape(1, -1))[0]

if prev_finished:
    players['predicted_score'] = retrieved_xgboost_model.predict(players[inputs]).astype("float64")
    print("made predictions")

made predictions


### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

### Create Score Graph
Draw a graph of the predictions with all the players as a PNG and save it to the github

In [431]:
if prev_finished:
    # Get or create feature group
    monitor_fg = fs.get_or_create_feature_group(
        name='fpl_predictions',
        description='FPL player score prediction monitoring',
        version=1,
        primary_key=['id', 'gameweek']
    )
    print("Got the monitor fg")

Got the monitor fg


In [432]:
if prev_finished:
    print("Setting the output data")
    # Only keep the relevant columns
    output_data = players[['id', 'points', 'gameweek', 'predicted_score', 'prev_total_points', 'prev_value', 'prev_opponent_team']]

Setting the output data


In [433]:
if prev_finished:
    # Add static data
    for i in range(0, len(output_data)):
        player_id = output_data.loc[i, 'id']

        player_data = util.get_player_info(player_id, general_info)
        for key in player_data:
            output_data.loc[i, key] = player_data[key]
    print("Added static data")

2025-01-07 13:08:11,178 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-01-07 13:08:11,180 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-01-07 13:08:11,181 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-01-07 13:08:11,182 WARNING: S

In [434]:
if prev_finished:
    monitor_fg.insert(output_data)
    print("Inserted the data")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1159321/fs/1150024/fg/1394714


Uploading Dataframe: 100.00% |██████████| Rows 484/484 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: fpl_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1159321/jobs/named/fpl_predictions_1_offline_fg_materialization/executions
Inserted the data


In [435]:
if prev_finished:
    monitor_fg.update_feature_description("id", "Player ID")
    monitor_fg.update_feature_description("points", "Total points of the player in the gameweek (label)")
    monitor_fg.update_feature_description("gameweek", "Gameweek")
    monitor_fg.update_feature_description("predicted_score", "Predicted score of the player in the gameweek")
    monitor_fg.update_feature_description("first_name", "Player first name")
    monitor_fg.update_feature_description("second_name", "Player surname")
    monitor_fg.update_feature_description("team", "Player team")
    monitor_fg.update_feature_description("position", "Player position")
    monitor_fg.update_feature_description("prev_total_points", "Total points of the player in the previous gameweek")
    monitor_fg.update_feature_description("prev_value", "Value of the player in the previous gameweek")
    monitor_fg.update_feature_description("prev_opponent_team", "Opponent team of the player in the previous gameweek")
    print("Updated the descriptions")

Updated the descriptions


---